In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, auc, f1_score
import tqdm
random.seed(42)
np.random.seed(42)

In [ ]:
df = pd.read_parquet('/kaggle/input/purple-hack/train_ai_comp_final_dp.parquet')
test = pd.read_parquet('/kaggle/input/purple-hack-with-test/test_sber.parquet')

In [ ]:
for col in df.columns:
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('int32')
    elif df[col].dtype == 'float64':
        df[col] = df[col].astype('float32')
    elif df[col].dtype == 'object':
        if len(df[col].unique()) / len(df[col]) < 0.5:
            df[col] = df[col].astype('category')

In [ ]:
# Удаление фичей, которые коррелируют друг с другом больше, чем на 0.9
def get_correlated_feats(corr_matrix, feat_stats, greater_is_better=True, corr_threshold=0.95):
    cols = corr_matrix.columns.to_list()
    dropped = {col:0 for col in cols}
    for col in tqdm.tqdm(cols, desc='Get correlated features'):
        if dropped[col] == 0:
            columns_to_check = corr_matrix.index.values[np.abs(corr_matrix[col]) >= corr_threshold]
            if len(columns_to_check) > 1:
                if feat_stats is None:
                    bad_cols = columns_to_check[1:]
                else:
                    sel_stats = feat_stats.loc[columns_to_check]
                    if greater_is_better:
                        bad_cond = np.abs(sel_stats) < np.abs(sel_stats).max()
                    else:
                        bad_cond = np.abs(sel_stats) > np.abs(sel_stats).min()
                        
                    bad_cols = sel_stats[bad_cond].index.to_list()
                    norm_cols = sel_stats[~bad_cond].index.to_list()
                    if len(norm_cols) > 1:
                        for norm_col in norm_cols[1:]:
                            dropped[norm_col] += 1
                            
                for bad_col in bad_cols:
                    dropped[bad_col] += 1
    high_corr_cols = [c for c in dropped.keys() if dropped[c] > 0]
    return high_corr_cols

In [ ]:
clean_df = df.copy()
nan_percentage = (clean_df == 0).mean()
cols_to_drop = list(nan_percentage[nan_percentage > 0.95].index)

ignore_features = ['id', 'target', 'sample_ml_new', 'feature756']+cols_to_drop
print(len(ignore_features))
clean_df = clean_df.drop(columns=ignore_features)

corr_mx = pd.DataFrame(np.corrcoef(clean_df.values, rowvar=False), columns=clean_df.columns, index=clean_df.columns)

In [ ]:
corr_feats = get_correlated_feats(corr_mx, feat_stats=nan_percentage, greater_is_better=False, corr_threshold=0.95)
len(corr_feats)

In [ ]:
df_no_corr = clean_df.drop(columns=corr_feats)
print("Было:", df.shape)
print("Стало:", df_no_corr.shape)

In [ ]:
final_feats = df_no_corr.columns

X = df[final_feats].drop(columns = ['feature642'])
y = df['target']

In [ ]:
X = X.reset_index(drop = True)
y = y.reset_index(drop = True)

In [ ]:
cat_cols = []
for col in X.columns:
    if len(X[col].value_counts()) < 3:
        cat_cols.append(col)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.25, random_state = 42)

In [ ]:
cat = CatBoostClassifier(random_seed = 42, class_weights = [1, 10])
cat.fit(X_train,y_train)

In [ ]:
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

top_10_features = importance_df.head(90)['Feature'].tolist()

X_train_top_90 = X_train[top_90_features]
X_test_top_90 = X_test[top_90_features]

In [ ]:
X_train_top_90['sin341'] = np.sin(X_train_top_90['feature341'])
X_train_top_90['log940'] = np.log1p(X_train_top_90['feature940'])
X_train_top_90['new1'] = (X_train_top_90['feature1004'] * X_train_top_90['feature994'])

X_test_top_90['sin341'] = np.sin(X_test_top_90['feature341'])
X_test_top_90['log940'] = np.log1p(X_test_top_90['feature940'])
X_test_top_90['new1'] = (X_test_top_90['feature1004'] * X_test_top_90['feature994'])